In [1]:
#hide
#default_exp tools.rerun
from nbdev.showdoc import *
from block_types.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Re-run previous experiments with new parameters

> Utility that allows to:
> - Perform more runs on previous experiments, each run having a different random seed. 
> - Increase the number of epochs used in previous experiments, when NN models are used.

In [2]:
#export
import argparse
from hpsearch.config.hpconfig import get_experiment_manager

In [3]:
#for tests
import pytest
from hpsearch.examples.complex_dummy_experiment_manager import generate_data, init_em

## parse_arguments_and_run

In [4]:
#export
def parse_args (args):
    parser = argparse.ArgumentParser(description='run experiment')
    parser.add_argument('-d', '--debug', action= "store_true")
    parser.add_argument('-e', '--experiments', type=int, nargs='+', default=None,  help="experiment numbers")
    parser.add_argument('--epochs', type=int, default=None,  help="number of epochs")
    parser.add_argument('-u', '--unfinished', action= "store_true")
    parser.add_argument('--runs', type=int, default=None,  help="number of runs")
    parser.add_argument('--root', type=str, default=None, help='name of root folder')
    pars = parser.parse_args(args)
    
    return pars

def parse_arguments_and_run (args, 
                             # for tests
                             em_args = None):
    
    pars = parse_args(args)

    other_parameters = dict(
                            use_process=not pars.debug,
                            root_folder=pars.root
                            )
    
    em = get_experiment_manager ()
    if em_args is not None:
        for k in em_args:
            setattr (em, k, em_args[k])
    
    if pars.epochs is not None:
        parameters = {em.name_epoch: int(pars.epochs)}
        other_parameters.update (prev_epoch=True)
        check_experiment_matches=False
    else:
        check_experiment_matches=True
        parameters = {}
    if pars.unfinished:
        other_parameters.update (check_finished=True, use_previous_best=False)
    
    
    
    em.rerun_experiment (experiments=pars.experiments, nruns=pars.runs, root_folder=pars.root, 
                         parameters=parameters, other_parameters=other_parameters, 
                         check_experiment_matches=check_experiment_matches)

def main():
    parse_arguments_and_run (sys.argv[1:])

### usage: performing more runs on previous experiments

In [5]:
#export tests.tools.test_rerun
def test_parse_arguments_and_run_more_runs ():
    em = generate_data ('parse_arguments_and_run_more_runs')

    df = em.get_experiment_data ()
    assert df.shape==(9,25)

    args = ['-e', '4', '3']
    parse_arguments_and_run (args)
    em.raise_error_if_run=True
    df = em.get_experiment_data ()
    x=[f'{i}_validation_accuracy' for i in range(5)]; assert df.columns.isin(x).sum()==5
    assert df.shape==(9,25)

    args = ['-e', '4', '3', '--runs', '10']
    em.raise_error_if_run=False
    parse_arguments_and_run (args)
    df = em.get_experiment_data ()
    assert df.shape==(9,45)
    x=[f'{i}_validation_accuracy' for i in range(10)]; assert df.columns.isin(x).sum()==10
    
    em.remove_previous_experiments ()

In [6]:
tst.run (test_parse_arguments_and_run_more_runs, tag='dummy', debug=False)

running test_parse_arguments_and_run_more_runs


### Increasing the number of epochs used in previous experiments

In [7]:
#export tests.tools.test_rerun
def test_parse_arguments_and_run_more_epochs ():
    em = init_em ('parse_arguments_and_run_more_epochs')

    # get reference result
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 7})
    df = em.get_experiment_data ()
    display (df)
    em.remove_previous_experiments()

    # first 3 experiments
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 5})
    _ = em.create_experiment_and_run (parameters={'offset':0.05, 'rate': 0.03, 'epochs': 6})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 9})
    _ = em.create_experiment_and_run (parameters={'offset':0.05, 'rate': 0.03, 'epochs': 10})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 11})
    df = em.get_experiment_data ()
    display (df)
    assert df.shape==(5,8)

    # more epochs
    #args = ['-e', '4', '3', '--epochs', '7', '-d']
    args = ['-e', '3', '--epochs', '7', '-d']
    parse_arguments_and_run (
        args, 
        em_args={'desired_path_results_previous_experiment':'test_parse_arguments_and_run_more_epochs/experiments/00001/0',
                 'desired_epochs': 1, 'desired_current_epoch': 7}
    )

    df = em.get_experiment_data ()
    print (df.shape)
    assert df.shape==(6,8)
    
    args = ['-e', '4', '--epochs', '7', '-d']
    parse_arguments_and_run (
        args, 
        em_args={'desired_path_results_previous_experiment':'test_parse_arguments_and_run_more_epochs/experiments/00000/0',
                 'desired_epochs': 2, 'desired_current_epoch': 7}
    )

    df = em.get_experiment_data ()
    print (df.shape)
    assert df.shape==(7,8)
    
    # *****************************************
    # *****************************************
    em.remove_previous_experiments ()
    em.desired_path_results_previous_experiment, em.desired_epochs, em.desired_current_epoch = None, None, None
    # first 3 experiments
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 5})
    _ = em.create_experiment_and_run (parameters={'offset':0.05, 'rate': 0.03, 'epochs': 6})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 9})
    _ = em.create_experiment_and_run (parameters={'offset':0.05, 'rate': 0.03, 'epochs': 10})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 11})
    
    args = ['-e', '4', '3', '--epochs', '7', '-d']
    parse_arguments_and_run (args)
    df = em.get_experiment_data ()
    print (df.shape)
    assert df.shape==(7,8)
    
    em.remove_previous_experiments ()

In [9]:
tst.run (test_parse_arguments_and_run_more_epochs, tag='dummy')

running test_parse_arguments_and_run_more_epochs
fitting model with 7 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996


,offset,rate,epochs,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,7.0,0.45,0.55,0.001563,11:59:35.955828,True


fitting model with 5 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
fitting model with 6 epochs
epoch 0: accuracy: 0.08
epoch 1: accuracy: 0.11
epoch 2: accuracy: 0.14
epoch 3: accuracy: 0.17
epoch 4: accuracy: 0.2
epoch 5: accuracy: 0.23
fitting model with 9 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
fitting model with 10 epochs
epoch 0: accuracy: 0.08
epoch 1: accuracy: 0.11
epoch 2: accuracy: 0.14
epoch 3: accuracy: 0.17
epoch 4: accuracy: 0.2
epoch 5: accuracy: 0.23
epoch 6: accuracy: 0.26
epoch 7: accuracy: 0.29000000000000004
epoch 8: accuracy: 0.32000000000000006
epoch 9: accuracy: 0.3500000000000001
fitting model with 11 epochs
epoch 0: ac

,offset,rate,epochs,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.10,0.05,5.0,0.35,0.45,0.001365,11:59:35.996519,True
1,0.05,0.03,6.0,0.23,0.33,0.001362,11:59:36.023671,True
2,0.10,0.05,9.0,0.55,0.65,0.001723,11:59:36.055353,True
3,0.05,0.03,NaN,0.35,0.25,0.002147,11:59:36.088853,True
4,0.10,0.05,11.0,0.65,0.55,0.001955,11:59:36.122707,True


reading model from test_parse_arguments_and_run_more_epochs/experiments/00001/0/model_weights.pk
fitting model with 1 epochs
epoch 0: accuracy: 0.26
(6, 8)
reading model from test_parse_arguments_and_run_more_epochs/experiments/00000/0/model_weights.pk
fitting model with 2 epochs
epoch 0: accuracy: 0.39999999999999997
epoch 1: accuracy: 0.44999999999999996
(7, 8)
fitting model with 5 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
fitting model with 6 epochs
epoch 0: accuracy: 0.08
epoch 1: accuracy: 0.11
epoch 2: accuracy: 0.14
epoch 3: accuracy: 0.17
epoch 4: accuracy: 0.2
epoch 5: accuracy: 0.23
fitting model with 9 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.549